Como Atividade M2, sugiro você repetir a Atividade M1, porém agora partindo da Atividade 3 da IC/TCC (onde dados in e out são complexos e há dados de extração e validação).

Importante:

1.o retorno da function (ou define) agora deve ser o módulo do vetor de erro (na Atividade M1 era apenas o vetor de erro). Ou seja, o retorno é um vetor de números reais;

2. sobre os argumentos da function, há 2 possibilidades:

2a) (essa possibilidade com certeza funciona) passar argumentos que são números reais que representam as partes real e imaginária de cada coeficiente (ou seja, se há C coeficientes complexos, passar 2C valores reais como argumento da function). Dentro da function você monta os coeficientes complexos. Nesse caso, o argumento e retorno da function são reais e, mesmo que dentro da function haja processamento com números complexos, isso não é "visível" para o otimizador;

2b) (não tenha certeza se funciona) passar argumentos para a function que são números complexos indicando diretamente os coeficientes complexos. Aqui tenho dúvidas se o otimizador irá otimizar ambas partes real e imaginária dos coeficientes.

3. Ao realizar o transposto, como agora os números são complexos, usar .' ou trans().

In [1]:
from scipy.io import loadmat

mat = loadmat('in_out_SBRT2_direto.mat')

in_data_ext = mat['in_extraction'].flatten()
out_data_ext = mat['out_extraction'].flatten()
in_data_val = mat['in_validation'].flatten()
out_data_val = mat['out_validation'].flatten()


In [2]:
import numpy as np

def erro_mp_complex(params, x_in, y_out, ordem, memoria):
    """
    params : vetor real (2*C), partes real e imaginária dos coef complexos
    x_in   : entrada complexa
    y_out  : saída desejada complexa
    ordem  : ordem do polinômio
    memoria: número de atrasos
    """
    # Reconstituir coeficientes complexos
    C = len(params)//2
    coef = params[:C] + 1j*params[C:]

    N = len(x_in)
    y_est = np.zeros(N, dtype=complex)

    for n in range(memoria, N):
        soma = 0
        idx = 0
        for m in range(memoria+1):
            for p in range(1, ordem+1):
                soma += coef[idx] * (x_in[n-m]**p)
                idx += 1
        y_est[n] = soma

    erro = y_out - y_est
    return np.abs(erro)   # retorno deve ser real (módulo do erro)


In [3]:
from scipy.optimize import least_squares

ordem = 3
memoria = 2
num_coef = ordem * (memoria+1)

# Vetor inicial: separar parte real e imaginária
x0 = np.zeros(2*num_coef)

res = least_squares(
    erro_mp_complex,
    x0,
    args=(in_data_ext, out_data_ext, ordem, memoria),
    verbose=2
)

coef_otimo = res.x[:num_coef] + 1j*res.x[num_coef:]
print("Coeficientes complexos otimizados:", coef_otimo)

# Calcular MSE na validação
erro_val = erro_mp_complex(res.x, in_data_val, out_data_val, ordem, memoria)
mse_val = np.mean(erro_val**2)
print("MSE validação:", mse_val)


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         8.8318e+02                                    2.03e+03    
       1              2         4.8194e+01      8.35e+02       1.00e+00       4.32e+02    
       2              3         8.9183e+00      3.93e+01       4.98e-01       7.93e+01    
       3              6         8.7755e+00      1.43e-01       8.28e-02       9.41e+01    
       4              7         6.8900e+00      1.89e+00       2.07e-02       1.39e+01    
       5              9         6.8643e+00      2.56e-02       5.17e-03       9.15e+00    
       6             10         6.8565e+00      7.85e-03       5.17e-03       9.37e+00    
       7             11         6.8440e+00      1.25e-02       1.29e-03       4.40e+00    
       8             12         6.8390e+00      4.99e-03       2.59e-03       3.82e+00    
       9             13         6.8359e+00      3.09e-03       2.59e-03       4.11e+00    

python : 
  1.23920092e+00+0.19266079j 
  1.33493909e-02-0.00757927j
  2.12979453e-03+0.00223977j
 -8.04116584e-01-0.29324995j
 -2.05289896e-02+0.00483429j
 -4.41562285e-04-0.00548455j
  4.47664061e-01+0.11310069j
  7.52697874e-03+0.00054942j
 -1.30981722e-03+0.00349838j

octave
   1.2780e+00 + 1.8317e-01i
   1.3679e-02 - 7.2619e-03i
   2.1350e-03 + 2.1053e-03i
  -8.8005e-01 - 2.7471e-01i
  -2.1048e-02 + 4.1201e-03i
  -4.3525e-04 - 5.2785e-03i
   4.8650e-01 + 1.0364e-01i
   7.7970e-03 + 9.5713e-04i
  -1.3024e-03 + 3.4079e-03i